In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import time
import pandas as pd
from keras.datasets import mnist
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_iris, make_circles, make_moons
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_curve, auc, f1_score, mean_squared_error, accuracy_score
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from google.colab import files
import io
from prophet import Prophet
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
building = pd.read_csv('/content/drive/MyDrive/building_info.csv', encoding = 'cp949')
test = pd.read_csv('/content/drive/MyDrive/test.csv', encoding = 'cp949')
train = pd.read_csv('/content/drive/MyDrive/train.csv', encoding = 'cp949')
sub = pd.read_csv('/content/drive/MyDrive/sample_submission.csv',encoding = 'cp949')

In [4]:
df = pd.DataFrame()
df['ds'] = train['일시']
df['y'] = train['전력소비량(kWh)']
df['ds'] = pd.to_datetime(df['ds'], format='%Y%m%d %H')


# 데이터프레임을 2040 단위로 나누어 리스트로 저장
dfs_list = [df[i:i+2040] for i in range(0, len(df), 2040)]

# dfs_list에 저장된 데이터프레임을 df100까지만 남기기
#df100 = pd.concat(dfs_list[:100], ignore_index=True)

In [5]:
len(dfs_list)

100

In [6]:
"""m = Prophet()
m.fit(df[:])
future = m.make_future_dataframe(freq='H',periods=24*7)
future.tail()"""

"m = Prophet()\nm.fit(df[:])\nfuture = m.make_future_dataframe(freq='H',periods=24*7)\nfuture.tail()"

In [7]:
"""forecast = m.predict(future)
forecast[['ds', 'yhat']].tail()"""

"forecast = m.predict(future)\nforecast[['ds', 'yhat']].tail()"

In [8]:

# dfs_list contains the list of data frames, and m is the initial Prophet model
dfs_list = [df[i:i+2040] for i in range(0, len(df), 2040)]

# List to store forecasts from each model
forecasts = []

# Iterate through each data frame in dfs_list and create a Prophet model
for df_chunk in dfs_list:
    m = Prophet()
    # Fit the model to the current data frame chunk
    m.fit(df_chunk)

    # Create a future dataframe for the next 7 days with hourly frequency
    future = m.make_future_dataframe(freq='H', periods=24*7)

    # Make predictions for the future dataframe
    forecast = m.predict(future)

    # Append the forecast to the list of forecasts
    forecasts.append(forecast)

# Now, 'forecasts' list will contain 100 forecasts, each corresponding to one chunk of data


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsodkwok3/eb3xi1_g.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsodkwok3/qqkwjqjm.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=16966', 'data', 'file=/tmp/tmpsodkwok3/eb3xi1_g.json', 'init=/tmp/tmpsodkwok3/qqkwjqjm.json', 'output', 'file=/tmp/tmpsodkwok3/prophet_modelqcq8xs9w/prophet_model-20230726013456.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:34:56 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:34:57 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/t

In [15]:
forecasts[0][2040:]['yhat']

2040    1854.980612
2041    1823.029764
2042    1763.558661
2043    1696.200066
2044    1676.475827
           ...     
2203    2843.586244
2204    2431.321701
2205    2108.602310
2206    1911.866542
2207    1825.212501
Name: yhat, Length: 168, dtype: float64

In [17]:
# Initialize an empty list to store the results
answer = []

# Loop through the forecasts list to extract 'yhat' values from each model
for forecast in forecasts:
    yhat_values = forecast[2040:]['yhat'].tolist()
    answer.extend(yhat_values)

# 'answer' now contains all the 'yhat' values from all the models
answer

[1854.980611652063,
 1823.029763908265,
 1763.5586606141635,
 1696.2000660406961,
 1676.4758274442383,
 1768.5513679240144,
 2014.287920978934,
 2411.2597154199875,
 2908.066407666445,
 3419.478781320202,
 3856.535595917362,
 4158.816585922862,
 4312.495329502854,
 4343.601228230466,
 4290.518000754366,
 4174.171390852158,
 3987.0884148169544,
 3709.450696521226,
 3339.904304735282,
 2916.758944782985,
 2511.391946286228,
 2196.0148759098383,
 2006.9999575863928,
 1928.3673424346725,
 1905.9337697790813,
 1882.328265957481,
 1831.2064447639027,
 1772.0846409121523,
 1760.3615835184453,
 1860.074351425365,
 2112.9521866649866,
 2516.4353555485736,
 3018.989333144026,
 3535.2523090822588,
 3976.1338736513153,
 4281.089861052664,
 4436.177045150753,
 4467.316905122583,
 4412.795839444893,
 4293.454505395439,
 4101.748586473287,
 3817.8035145234267,
 3440.225541177767,
 3007.3019923402394,
 2590.4061311425276,
 2261.7644441249604,
 2057.7835211542024,
 1962.5375621791368,
 1921.91704548403

In [18]:
sub['answer'] = answer
sub

,num_date_time,answer
0,1_20220825 00,1854.980612
1,1_20220825 01,1823.029764
2,1_20220825 02,1763.558661
3,1_20220825 03,1696.200066
4,1_20220825 04,1676.475827
...,...,...
16795,100_20220831 19,989.487761
16796,100_20220831 20,908.439229
16797,100_20220831 21,804.232654
16798,100_20220831 22,680.273344


In [19]:
sub.to_csv('./prophet_submission.csv', index=False)